In [1]:
from smolagents import CodeAgent, InferenceClientModel, Tool, DuckDuckGoSearchTool
from dotenv import load_dotenv
import os
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from langfuse import get_client
from huggingface_hub import notebook_login
notebook_login()
load_dotenv()
angfuse_key = os.getenv('ANGFUSE_SECRET_KEY')
model_id = "Qwen/Qwen2.5-14B-Instruct"

In [2]:
class RecipeRetrieverTool(Tool):
    name = "retrieve_recipe"
    description = "Retrieves the best matching recipe/s from the database. Returns raw title and ingredients."
    inputs = {
        "query": {"type": "string", "description": "Dish name or ingredients."},
    }
    
    output_type = "string"
    
    def __init__(self, k: int = 1, **kwargs):
        """Initialize the recipe retrieval tool.
        
        Args:
            k: Number of top recipes to retrieve (default: 1)
        """
        super().__init__(**kwargs)
        self.k = k
        
        print(f"Loading recipe retrieval system with top {k} results")
        
        # Load metadata from embeddings file
        self.metadata = []
        with open('recipes_for_embeddings.jsonl', 'r') as f:
            for line in f:
                self.metadata.append(json.loads(line))
        
        # Load full recipe details (with ingredients and directions)
        with open('full_format_recipes.json', 'r') as f:
            full_recipes = json.load(f)
        
        # HashTable for optimized lookup
        self.recipe_lookup = {r.get('title', '').strip(): r for r in full_recipes if r.get('title')}
        
        # Load embedding model
        print("Loading embedding model BAAI/bge-m3...")
        self.embed_model = SentenceTransformer('BAAI/bge-m3')
        
        # Load FAISS index
        print("Loading FAISS index...")
        self.index = faiss.read_index('recipe_index.faiss')
        
        print(f"✓ Recipe retrieval system loaded: {len(self.metadata)} recipes indexed")
    
    def forward(self, query: str) -> str:
        """Search for recipes matching the query.
        
        Args:
            query: Natural language search query
            
        Returns:
            Formatted string with recipe titles, ingredients, and directions
        """
        assert isinstance(query, str), "Query must be a string."
        
        # 1. Embed the Query and Ensure Float 32 (Required by FAISS)
        query_vec = self.embed_model.encode([query], convert_to_tensor=False)
        query_vec = np.array(query_vec).astype('float32')
        
        # 2. Normalize euclidean distance
        faiss.normalize_L2(query_vec)
        
        # 3. Search
        _, indices = self.index.search(query_vec, self.k)
        
        # 4. Retrieve
        retrieved_docs = [self.metadata[idx] for idx in indices[0] if idx != -1]
        
        # 5. Format Output
        output = f"Found {len(retrieved_docs)} recipes matching '{query}':\n\n"
        
        for i, doc in enumerate(retrieved_docs, 1):
            title = doc['title'].strip()
            full_recipe = self.recipe_lookup.get(title)
            
            output += f"{'='*40} Recipe {i} {'='*40}\n"
            output += f"TITLE: {title}\n"
            
            if full_recipe:
                output += "INGREDIENTS:\n" + "\n".join([f" - {ing}" for ing in full_recipe.get('ingredients', [])])
                output += "\n\nDIRECTIONS:\n" + "\n".join([f" {j}. {step}" for j, step in enumerate(full_recipe.get('directions', []), 1)])
            else:
                # Fallback
                output += f"SUMMARY: {doc.get('text_for_embedding', 'No details available')}"
            
            output += "\n\n"
            
        return output

In [3]:
class RecipeAdapterTool(Tool):
    name = "adapt_recipe"
    description = "Rewrites a recipe to comply with a specific dietary constraint (e.g., 'Make this vegan')."
    inputs = {
        "recipe_text": {"type": "string", "description": "The original recipe text."},
        "target_diet": {"type": "string", "description": "The target diet (e.g., 'vegan', 'gluten-free')."}
    }
    output_type = "string"

    def __init__(self, model_engine, **kwargs):
        super().__init__(**kwargs)
        self.model_engine = model_engine

    def forward(self, recipe_text: str, target_diet: str) -> str:
        # We construct a prompt for the LLM to do the rewriting
        prompt = f"""
        You are an expert chef. Rewrite the following recipe to be strictly {target_diet}.
        
        Rules:
        1. Replace ONLY forbidden ingredients with best culinary substitutes.
        2. Keep the original formatting.
        3. Do not change the dish identity (e.g., 'Beef Stew' becomes 'Lentil Stew', not 'Salad').
        
        Original Recipe:
        {recipe_text}
        
        Rewritten Recipe:
        """
        
        # Call the LLM (using smolagents' model wrapper)
        messages = [{"role": "user", "content": prompt}]
        return self.model_engine(messages)

In [4]:
class RecipeValidatorTool(Tool):
    name = "validate_recipe"
    description = "Analyzes a recipe to check if it strictly follows a dietary constraint."
    inputs = {
        "recipe_text": {"type": "string", "description": "The recipe to check."},
        "constraint": {"type": "string", "description": "The constraint (e.g., 'vegan')."}
    }
    output_type = "string"

    def __init__(self, model_engine, **kwargs):
        super().__init__(**kwargs)
        self.model_engine = model_engine

    def forward(self, recipe_text: str, constraint: str) -> str:
        prompt = f"""
        Analyze this recipe for compliance with: {constraint}.
        
        Recipe:
        {recipe_text}
        
        Task:
        Check for any hidden forbidden ingredients (e.g., 'whey' in vegan, 'soy sauce' in gluten-free).
        
        Output format:
        STATUS: [PASS/FAIL]
        REASON: [Explanation]
        """
        
        messages = [{"role": "user", "content": prompt}]
        return self.model_engine(messages)

## Instantiate the model

In [5]:
model = InferenceClientModel(model_id=model_id)

# Instantiate the tools

In [7]:
retriever_tool = RecipeRetrieverTool()
adapter_tool = RecipeAdapterTool(model_engine=model)
validator_tool = RecipeValidatorTool(model_engine=model)
search_tool = DuckDuckGoSearchTool()

Loading recipe retrieval system with top 1 results
Loading embedding model BAAI/bge-m3...
Loading FAISS index...
✓ Recipe retrieval system loaded: 18222 recipes indexed


# Define prompt for the multiagent

In [8]:
SYSTEM_PROMPT = """
You are an intelligent Culinary Agent powered by Qwen-72B.
Your goal is to find, adapt, and validate recipes to strictly meet the user's needs.

AVAILABLE TOOLS:
1. `retrieve_recipe(query)`: Searches the internal database.
2. `adapt_recipe(recipe_text, target_diet)`: Rewrites a recipe to be compliant.
3. `validate_recipe(recipe_text, constraint)`: Checks for safety/compliance.
4. `duckduckgo_search(query)`: Searches the web for unknown ingredients.

STANDARD OPERATING PROCEDURE (SOP):
1. **INITIAL SEARCH**: Always start by calling `retrieve_recipe` with the user's dish name.
   
2. **ANALYSIS & CHECK**: 
   - Read the ingredients of the retrieved recipe.
   - Does it meet the user's dietary constraint (e.g., vegan, gluten-free)?
   - If YES: Proceed to Step 5 (Validation).
   - If NO: Proceed to Step 3.

3. **EXPANDED SEARCH (Fallback)**:
   - If the first recipe failed the check, run `retrieve_recipe` again, but this time modify the query to include the diet (e.g., "vegan lasagna").
   - Check the new results.
   - If a compliant recipe is found, use it.
   - If STILL no compliant recipe is found, take the *best available* non-compliant recipe and proceed to Step 4.

4. **ADAPTATION**:
   - Call `adapt_recipe` on the non-compliant recipe text.
   - *CRITICAL*: If you encounter an ingredient you are unsure about (e.g., "Is Worcestershire sauce vegan?"), first call `duckduckgo_search` to verify, then adapt accordingly.

5. **FINAL VALIDATION**:
   - Before answering the user, you MUST run `validate_recipe` on your final candidate.
   - If it returns "FAIL", loop back to Step 4 (Adapt) to fix the specific error mentioned.
   - If it returns "PASS", present the final recipe to the user.

OUTPUT FORMAT:
- When you are thinking, use the python code block to execute tools.
- When you have the final "PASS" result, provide the recipe clearly to the user.
"""

# Instantiate the multiagent

In [11]:
agent = CodeAgent(
    tools=[
        retriever_tool, 
        adapter_tool, 
        validator_tool, 
        search_tool
    ],
    model=model,
    add_base_tools=True, # This enables the "scratchpad" where Qwen writes its Python logic
    max_steps=12, # Give it enough turns to think -> adapt -> validate
    )


In [12]:
def run_agent(user_query: str) -> str:
    full_prompt = f"""You are an intelligent cullinary agent. Follow this SOP restrictly:
    {SYSTEM_PROMPT}
    USER REQUEST: {user_query}
    """
    return agent.run(full_prompt)
    

In [13]:
response = run_agent("I need a high protein recipe that includes chicken.")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an intelligent cullinary agent. Follow this SOP restrictly:                                             │
│                                                                                                                 │
│ You are an intelligent Culinary Agent powered by Qwen-72B.                                                      │
│ Your goal is to find, adapt, and validate recipes to strictly meet the user's needs.                            │
│                                                                                                                 │
│ AVAILABLE TOOLS:                                                                                                │
│ 1. `retrieve_recipe(query)`: Searches the internal database.                                                    │
│ 2. `adapt_recipe(recipe_text, target_diet)`: Rewrites a recipe to be compliant.                                 │
│ 3. `validate_recipe(recipe_text, constraint)`: Checks for safety/compliance.                                    │
│ 4. `duckduckgo_search(query)`: Searches the web for unknown ingredients.                                        │
│                                                                                                                 │
│ STANDARD OPERATING PROCEDURE (SOP):                                                                             │
│ 1. **INITIAL SEARCH**: Always start by calling `retrieve_recipe` with the user's dish name.                     │
│                                                                                                                 │
│ 2. **ANALYSIS & CHECK**:                                                                                        │
│    - Read the ingredients of the retrieved recipe.                                                              │
│    - Does it meet the user's dietary constraint (e.g., vegan, gluten-free)?                                     │
│    - If YES: Proceed to Step 5 (Validation).                                                                    │
│    - If NO: Proceed to Step 3.                                                                                  │
│                                                                                                                 │
│ 3. **EXPANDED SEARCH (Fallback)**:                                                                              │
│    - If the first recipe failed the check, run `retrieve_recipe` again, but this time modify the query to       │
│ include the diet (e.g., "vegan lasagna").                                                                       │
│    - Check the new results.                                                                                     │
│    - If a compliant recipe is found, use it.                                                                    │
│    - If STILL no compliant recipe is found, take the *best available* non-compliant recipe and proceed to Step  │
│ 4.                                                                                                              │
│                                                                                                                 │
│ 4. **ADAPTATION**:                                                                                              │
│    - Call `adapt_recipe` on the non-compliant recipe text.                                                      │
│    - *CRITICAL*: If you encounter an ingredient you are unsure about (e.g., "Is Worcestershire sauce vegan?"),  │
│ first call `duckduckgo_search` to verify, then adapt accordingly.                                               │
│                                                       

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  recipe_text = retrieve_recipe(query="high protein chicken recipe")                                               
  print(recipe_text)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Found 1 recipes matching 'high protein chicken recipe':

======================================== Recipe 1 ========================================
TITLE: Spitted Roast Chicken
INGREDIENTS:


DIRECTIONS:
 1. Put a good square of butter and a little salt and pepper in the cavity of each chicken. Truss well and brush 
them with melted butter or oil seasoned to taste with salt, pepper and paprika. Spit them carefully. Run the spit 
through the backbone just above the tail and guide it to the top part of the breast at the base of the neck. This 
way you achieve a good balance.
 2. When your fire has burned down to a good bed of coals, make a ring of the briquets or charcoal leaving the 
center area directly under the chickens clear to catch the drippings. Arrange the spitted chickens over this space 
and roast, basting them frequently with equal parts of melted butter and white wine or dry vermouth. The cooking 
time will take from 45 minutes to 1 1/4 hours, depending on the size of the birds.
 3. Plain roast chicken goes best with crisp sautéed potatoes and a fresh green salad with a minimum of dressing. 
As for wine, most people prefer a white, such as a Pinot Blanc from California, or a Meursault or Pouilly Fuissé 
from France.



Out: None

[Step 1: Duration 9.29 seconds| Input tokens: 2,755 | Output tokens: 45]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(recipe_text)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Found 1 recipes matching 'high protein chicken recipe':

======================================== Recipe 1 ========================================
TITLE: Spitted Roast Chicken
INGREDIENTS:


DIRECTIONS:
 1. Put a good square of butter and a little salt and pepper in the cavity of each chicken. Truss well and brush 
them with melted butter or oil seasoned to taste with salt, pepper and paprika. Spit them carefully. Run the spit 
through the backbone just above the tail and guide it to the top part of the breast at the base of the neck. This 
way you achieve a good balance.
 2. When your fire has burned down to a good bed of coals, make a ring of the briquets or charcoal leaving the 
center area directly under the chickens clear to catch the drippings. Arrange the spitted chickens over this space 
and roast, basting them frequently with equal parts of melted butter and white wine or dry vermouth. The cooking 
time will take from 45 minutes to 1 1/4 hours, depending on the size of the birds.
 3. Plain roast chicken goes best with crisp sautéed potatoes and a fresh green salad with a minimum of dressing. 
As for wine, most people prefer a white, such as a Pinot Blanc from California, or a Meursault or Pouilly Fuissé 
from France.

[Step 4: Duration 3.15 seconds| Input tokens: 14,280 | Output tokens: 273]

Found 1 recipes matching 'high protein chicken recipe':

======================================== Recipe 1 ========================================
TITLE: Spitted Roast Chicken
INGREDIENTS:


DIRECTIONS:
 1. Put a good square of butter and a little salt and pepper in the cavity of each chicken. Truss well and brush them with melted butter or oil seasoned to taste with salt, pepper and paprika. Spit them carefully. Run the spit through the backbone just above the tail and guide it to the top part of the breast at the base of the neck. This way you achieve a good balance.
 2. When your fire has burned down to a good bed of coals, make a ring of the briquets or charcoal leaving the center area directly under the chickens clear to catch the drippings. Arrange the spitted chickens over this space and roast, basting them frequently with equal parts of melted butter and white wine or dry vermouth. The cooking time will take from 45 minutes to 1 1/4 hours, depending on the size of the birds.
 